In [ ]:
!pip install gradio
import gradio as gr

In [ ]:
!pip install openai
import openai
openai.api_key = 'OpenAIAPI金鑰'   # 金鑰

In [4]:
dialogue = []      # 對話記錄串列
def get_response(dialogue, streamTF=False):
   response = openai.chat.completions.create(
       model = 'gpt-3.5-turbo',
       messages = dialogue,
       stream = streamTF                             # 設定流式傳輸生成回覆
   )
   if streamTF:
     for chunk in response:                         # response 為可迭代物件
       if chunk.choices[0].delta.content is not None:   # 迭代元素時,防止None文本
         yield chunk.choices[0].delta.content         # 傳出片段回覆文本
   else:
     yield response.choices[0].message.content    # 傳出所有回覆文本

In [5]:
saveTalk = 2       # 保留最後2組提問和回覆記錄
def chat(ask, streamTF):
  global dialogue            # 宣告dialogue屬於全域變數
  dialogue.append({'role':'user','content':ask})       # 存放提問資料
  ans_all = ''
  for answer in get_response(dialogue, streamTF):     # 呼叫函式生成片段文本
    ans_all += answer
    yield answer
  dialogue.append({'role':'assistant','content':ans_all})  # 新增元素
  while len(dialogue) > saveTalk * 2:   # 當提問和回覆記錄超過2組時
    dialogue.pop(0)                   # 把前面的串列元素移除掉

In [6]:
def gather_chat(ask, streamTF):
  sentence = ''
  for ans in chat(ask, streamTF):
    sentence += ans            # 累加片段文句
    yield sentence

In [ ]:
txtAsk = gr.Textbox(label='輸入您的問題：')
chkStream = gr.Checkbox(label='流式傳輸', value=False)
gr.Interface(
    fn = gather_chat,
    inputs = [txtAsk, chkStream],
    outputs= gr.Textbox(label='AI 的回答：'),
    title = 'AI 聊天室'
).queue().launch(share=True)

In [ ]:
dialogue = []            # 移除串列所有元素內容